In [1]:
!pip install pdf2image --upgrade

In [2]:
!sudo apt-get install poppler-utils
!pip install -U spacy -q

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 1s (299 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 120874 

In [3]:
from pdf2image import convert_from_path
from PIL import Image
import os
import io
import pandas as pd
import numpy as np
import spacy
# from paddleocr import PaddleOCR
from spacy.tokens import DocBin
import json
from tqdm import tqdm
nlp = spacy.blank("en") # load a new spacy model
db = DocBin()
import re

In [55]:
import json
f = open('/content/DISTRIBUTION_ANNOTATION.json')

TRAIN_DATA = json.load(f)

UnicodeDecodeError: ignored

In [20]:
# text=TRAIN_DATA['annotations'][0]
# text
TRAIN_DATA['annotations'][0][1]

{'entities': [[0, 29, 'FUND_NAME'],
  [93, 107, 'AMOUNT_DUE'],
  [122, 137, 'NOTICE_DATE'],
  [152, 165, 'DUE_DATE'],
  [187, 200, 'RETURN_OF_CAPITAL'],
  [211, 224, 'RETURN_OF_CAPITAL'],
  [235, 248, 'RETURN_OF_CAPITAL'],
  [353, 367, 'AMOUNT_DUE'],
  [591, 601, 'LP_NAME'],
  [602, 614, 'RETURN_OF_CAPITAL'],
  [642, 652, 'LP_NAME'],
  [653, 666, 'RETURN_OF_CAPITAL'],
  [695, 705, 'LP_NAME'],
  [706, 718, 'RETURN_OF_CAPITAL'],
  [746, 756, 'LP_NAME'],
  [757, 769, 'RETURN_OF_CAPITAL'],
  [797, 807, 'LP_NAME'],
  [808, 820, 'RETURN_OF_CAPITAL'],
  [847, 857, 'LP_NAME'],
  [858, 870, 'RETURN_OF_CAPITAL'],
  [898, 908, 'LP_NAME'],
  [909, 921, 'RETURN_OF_CAPITAL'],
  [949, 959, 'LP_NAME'],
  [960, 973, 'RETURN_OF_CAPITAL'],
  [1002, 1012, 'LP_NAME'],
  [1013, 1023, 'RETURN_OF_CAPITAL'],
  [1061, 1074, 'RETURN_OF_CAPITAL'],
  [1089, 1103, 'AMOUNT_DUE']]}

In [21]:
def create_training_data(training_data, output_dir):
    nlp = spacy.blank("en")
    db = DocBin()

    for text, annot in tqdm(training_data['annotations']):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)

    db.to_disk(output_dir)
create_training_data(TRAIN_DATA, "./training_data.spacy")


100%|██████████| 45/45 [00:00<00:00, 1037.51it/s]

Skipping entity


In [22]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

2023-11-01 05:10:43.912394: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-01 05:10:43.912458: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-01 05:10:43.912497: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-01 05:10:45.145554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.

2023-11-01 05:10:53.432023: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: 

In [62]:
file_path = "/content/EQT X   Distribution   2023 07 14 (1).json"

In [27]:
headings = {
    # "income_statement": {
    #     "label": "Income Statement",
    #     "pattern": ["Income statement", "Income summary", "Income and Expenses", "Income & Expense", "Cash Flow", "Budget Comparison"],
    #     "tags": ['ASSETNAME', 'DOCUMENTNAME', 'DOCUMENTPERIOD', 'MONTH1', 'MONTH2', 'MONTH3', 'MONTH4', 'MONTH5', 'MONTH6', 'MONTH7', 'MONTHS', 'MONTH9', 'MONTH10', 'MONTH11', 'MONTH12', 'ACCOUNT', 'AMOUNTMONTH1', 'AMOUNTMONTH2', 'AMOUNTMONTH3', 'AMOUNTMONTH4', 'AMOUNTMONTHS', 'AMOUNTMONTH6', 'AMOUNTMONTH7', 'AMOUNTMONTHS', 'AMOUNTMONTH9', 'AMOUNTMONTH10', 'AMOUNTMONTH11', 'AMOUNTMONTH12', 'AMOUNTTOTAL']
    # },
    # "rent_roll": {
    #     "label": "Rent Roll",
    #     "pattern": ["Rent roll", "Rentroll"],
    #     "tags": ['UNIT_NUMBER', 'UNIT_TYPE', 'TENANT_NAME', 'SPACE_SIZE', 'SPACE_SIZE_UNIT_TYPE', 'MARKET_RENT', 'RENTAL_AMOUNT', 'RENTAL_PERIOD', 'LEASE_DATE', 'MOVE_IN_DATE', 'LEASE_EXPIRATION_DATE', 'MOVE_OUT_DATE', 'BASE_RENT_PERIOD', 'RENT_START_DATE', 'VACANCY_RATE', 'OCCUPANCY_RATE', 'UNITS_AVAILABLE', 'VACANT_UNITS', 'OCCUPIED_UNITS', 'NUMBER_OF_ASSIGNED_UNITS', 'AVERAGE_SPACE_SIZE', 'ASSIGNED_UNIT_TYPE', 'BASE_RENT']
    # },
    "capital": {
        "label": "Capital",
        "pattern": ["capital", "Capital","CAPITAL"],
        "tags":['FUND_NAME',
 'GP_NAME',
 'LP_NAME',
 'INVESTMENT_NAME',
 'INVESTMENT_AMOUNT',
 'NOTICE_DATE',
 'DUE_DATE',
 'CURRENCY_CODE',
 'AMOUNT_DUE',
 'COMMITMENT_AMOUNT',
 'UNFUNDED_COMMITMENT',
 'INVESTOR_NUMBER',
 'NOTICE_NUMBER',
 'INVESTMENT_CALL_AMOUNT',
 'EXCESS_CASH_RETURNED',
 'MANAGEMENT_FEE',
 'MANAGEMENT_FEE_DISCOUNT',
 'MAKE_UP_MANAGEMENT_FEE',
 'MANAGEMENT_FEE_INTEREST',
 'NET_MANAGEMENT_FEE',
 'PARTNERSHIP_EXPENSES',
 'ORGANIZATIONAL_COSTS',
 'DISTRIBUTION_OF_PROCEEDS',
 'CAPITAL_CONTRIBUTION_',
 'AGGREGATE_CAPITAL_CALL_AMOUNT',
 'TOTAL_MAKE_UP_CONTRIBUTIONS',
 'MAKE_UP_INTEREST_PAYMENT',
 'UNDRAWN_COMMITMENT_BEFORE',
 'UNDRAWN_COMMITMENT_EFFECT',
 'UNDRAWN_COMMITMENT_AFTER'],
    },
    "distribution":{
        "label":"Distribution",
        "pattern":["Distribution","distribution","DISTRIBUTION"],
        "tags":['FUND_NAME',
 'GP_NAME',
 'LP_NAME',
 'INVESTMENT_NAME',
 'INVESTMENT_AMOUNT',
 'NOTICE_DATE',
 'DUE_DATE',
 'CURRENCY_CODE',
 'AMOUNT_DUE',
 'COMMITMENT_AMOUNT',
 'UNFUNDED_COMMITMENT',
 'INVESTOR_NUMBER',
 'NOTICE_NUMBER',
 'INVESTMENT_CALL_AMOUNT',
 'EXCESS_CASH_RETURNED',
 'MANAGEMENT_FEE',
 'MANAGEMENT_FEE_DISCOUNT',
 'MAKE_UP_MANAGEMENT_FEE',
 'MANAGEMENT_FEE_INTEREST',
 'NET_MANAGEMENT_FEE',
 'PARTNERSHIP_EXPENSES',
 'ORGANIZATIONAL_COSTS',
 'DISTRIBUTION_OF_PROCEEDS',
 'CAPITAL_CONTRIBUTION_',
 'AGGREGATE_CAPITAL_CALL_AMOUNT',
 'TOTAL_MAKE_UP_CONTRIBUTIONS',
 'MAKE_UP_INTEREST_PAYMENT',
 'UNDRAWN_COMMITMENT_BEFORE',
 'UNDRAWN_COMMITMENT_EFFECT',
 'UNDRAWN_COMMITMENT_AFTER'],
    },
}

In [57]:
def create_image(pdf_path):
    pages = []
    images = convert_from_path(pdf_path)
    total_pages = len(images)
    print("Total Pages:", total_pages)

    # Define the folder name you want to create
    folder_name = 'Pages'

    # Create the folder in the current working directory
    folder_path = os.path.join('/content', folder_name)

    try:
        os.makedirs(folder_path)
        print("Folder created successfully!")
    except FileExistsError:
        print("Folder already exists.")

    # Saving pages as image
    for i in range(total_pages):
        page_path = os.path.join(folder_path, f'page{i}.jpg')
        pages.append(page_path)
        print("Page Paths:", pages)
        images[i].save(page_path, 'JPEG')

    return pages
# if file_path.endswith('.pdf'):
#    create_image(file_path)


In [51]:
def delete_image(pages):
    for page in pages:
        try:
            os.remove(page)
            print(f"Deleted {page}")
        except FileNotFoundError:
            print(f"{page} not found.")
        except Exception as e:
            print(f"Failed to delete {page}: {e}")
    pages = []

In [60]:
def ner(necessary_data):
    nlp_ner = spacy.load("/content/model-best")
    doc= nlp_ner(necessary_data[0])
    print("doc--------->",doc,"-------------------->docent")
    print("data----------?>",necessary_data)

    spacy.displacy.render(doc, style="ent", jupyter=True)
    entities_table = []
    for ent in doc.ents:
        if ent.label_ in necessary_data[1]:
            entities_table.append([ent.text, ent.label_])
            # print("Start Char Position:", ent.start_char)
            # print("End Char Position:", ent.end_char)
    print("ner--------------------->",entities_table)

    # Print the entities and labels  in a table format
    df = pd.DataFrame(entities_table, columns=["Entity Text", "Entity Label"])
    df.columns = ["Text", "Label"]
    return df

In [45]:
%pip install  paddlepaddle-gpu paddleocr>=2.0.1 -q

!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb # this
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb # and this is needed for paddle ocr

--2023-11-01 05:32:08--  http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb
Resolving nz2.archive.ubuntu.com (nz2.archive.ubuntu.com)... 91.189.91.83, 185.125.190.39, 91.189.91.82, ...
Connecting to nz2.archive.ubuntu.com (nz2.archive.ubuntu.com)|91.189.91.83|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-11-01 05:32:08 ERROR 404: Not Found.

dpkg: error: cannot access archive 'libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb': No such file or directory


In [46]:
from paddleocr import PaddleOCR

In [47]:
# extracting text from images
def read_from_pdf(pdf_path):
    # print("c")
    tags_page_wise = []
    pocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=False)
    entire_text = ""
    pages = create_image(pdf_path)
    print("PAGES",pages)
    for i, img_path in enumerate(pages):
        struct = pocr.ocr(img_path)
        struct = struct[0]
        text = ""
        for lst in struct:
            text += lst[1][0]
            text += " "


        found = False
        for key, val in headings.items():
            # print(key, val["pattern"])
            pattern_text = val["pattern"]
            for find_text in pattern_text:
                pattern = re.compile(r"\b" + re.escape(find_text) + r"\b", re.IGNORECASE)

                x = re.search(pattern, text)

                if x:
                    found = True
                    df = pd.DataFrame()
                    if i == 0:
                        parent_heading = val["label"]
                        print(f"\n\nDocument Type: \n{parent_heading} At Page: 1\n\n")
                        df = ner((text, val["tags"]))
                        print(df)
                    else:
                        child_heading = val["label"]
                        print(f"\n\nDocument Type: \n{parent_heading}-{child_heading} At Page: {i+1}\n\n")
                        df = ner((text, val["tags"]))
                    break

        if not found:
            print(f"\n\nHeading: \n{parent_heading}-MISC At Page: {i+1}\n\n")

        if len(df) != 0:
            df.to_csv(f"output_{i}.csv", index=False)  # Set index=False to exclude the index column
        else:
            print("DataFrame is Empty!")
        return text,df

    delete_image(pages)

In [24]:
def read_from_json(file_path):
    file = open(file_path)
    data = json.load(file)

    # extracting neccessary ids
    neccessary_ids = []

    for obj in data["Blocks"]:
        if obj["BlockType"] == "PAGE":
            for ind, ids in enumerate(obj["Relationships"][0]["Ids"]):
                neccessary_ids.append(ids)

    # print(neccessary_ids)
    text_in_doc = []
    text_in_page = ""
    cnt = 0
    for obj in data["Blocks"]:
        same_page = obj["Page"]
        if obj["Id"] in neccessary_ids and same_page == obj["Page"]:
            if "Text" in obj.keys():
                text_in_page += obj["Text"]
                text_in_page += " "
        elif same_page == obj["Page"]:
            same_page = obj["Page"]
            if text_in_page != "":
                text_in_doc.append(text_in_page)
            text_in_page = ""
    # print(len(text_in_doc))

    for i, text in enumerate(text_in_doc):
        found = False
        # print(obj["Text"],"At Page", obj["Page"])
        for key, val in headings.items():
            pattern_text = val["pattern"]
            for find_text in pattern_text:
                pattern = re.compile(r"\b" + re.escape(find_text) + r"\b", re.IGNORECASE)
                x = re.search(pattern, text)
                if x:
                    found = True
                    heading = val["label"]
                    tags = val["tags"]
                    print(f"\n\nHeading: {heading} At Page {i+1}\n\n")
                    df = ner((text, val["tags"]))
                    break
        if not found:
            print(f"\n\nHeading: \nMISC At Page {i+1}\n\n")

        if len(df) != 0:
            df.to_csv(f"output_{i}.csv", index=False)  # Set index=False to exclude the index column
        else:
            print("DataFrame is Empty!")

In [63]:
def extract_useful_data(file_path):
    if file_path.endswith('.json'):
        read_from_json(file_path)
    elif file_path.endswith('.pdf'):
        read_from_pdf(file_path)
    else:
        print("File Type Not Supported!")

In [64]:
# file_path = "/content/Return of Capital on 2023 08 04 for Vista Equity Partners Fund VIII, L.P..pdf"
extract_useful_data(file_path)
# print("the text is",text)



Heading: Capital At Page 1


doc---------> EGT EQT Fund Management S.àr.l. 51A, Boulevard Royal, L-2449 Luxembourg, Luxembourg STRICTLY PRIVATE AND CONFIDENTIAL NEWVEST VINTAGE 2022 AGGREGATOR PE 50 LP 14 July 2023 EQT X (No.2) USD SCSp (the "Partnership") Investor Reference: EQTXUSDII048 Notice of Capital Event: Investments Dear EQT Investor, Please be advised that this is a notice of a net zero event in connection with the investment in Billtrust for value date 18 July 2023. A net zero event implies that we are drawing and distributing the same amount, hence this notice is being provided to you for information purposes only. Therefore, as an investor you are not required to remit any contribution and no proceeds will be distributed in relation to this notice. Your share of the amount under the notice is detailed below: INVESTOR'S SUMMARY USD Billtrust - Equity Investment (28,166.48) Total Drawdown - Affecting Commitments (28,166.48) Billtrust - Return of Capital (Recallable) 28,166

ner---------------------> [['EGT EQT Fund Management S.àr.l.', 'FUND_NAME'], ['NEWVEST VINTAGE 2022 AGGREGATOR PE 50 LP', 'LP_NAME'], ['14 July 2023', 'NOTICE_DATE'], ['EQT X (No.2) USD SCSp (the "Partnership") Investor Reference: EQTXUSDII048 Notice of Capital Event: Investments Dear EQT Investor, Please be advised that this is a notice of a net zero event in connection with the investment in Billtrust for value date 18', 'FUND_NAME'], ['28,166.48', 'INVESTMENT_AMOUNT'], ['28,166.48', 'AMOUNT_DUE'], ['5,000,000.00', 'COMMITMENT_AMOUNT'], ['4,416,007.41', 'UNDRAWN_COMMITMENT_BEFORE'], ['4,416,007.41', 'UNFUNDED_COMMITMENT']]


Heading: Distribution At Page 1


doc---------> EGT EQT Fund Management S.àr.l. 51A, Boulevard Royal, L-2449 Luxembourg, Luxembourg STRICTLY PRIVATE AND CONFIDENTIAL NEWVEST VINTAGE 2022 AGGREGATOR PE 50 LP 14 July 2023 EQT X (No.2) USD SCSp (the "Partnership") Investor Reference: EQTXUSDII048 Notice of Capital Event: Investments Dear EQT Investor, Please be advi

ner---------------------> [['EGT EQT Fund Management S.àr.l.', 'FUND_NAME'], ['NEWVEST VINTAGE 2022 AGGREGATOR PE 50 LP', 'LP_NAME'], ['14 July 2023', 'NOTICE_DATE'], ['EQT X (No.2) USD SCSp (the "Partnership") Investor Reference: EQTXUSDII048 Notice of Capital Event: Investments Dear EQT Investor, Please be advised that this is a notice of a net zero event in connection with the investment in Billtrust for value date 18', 'FUND_NAME'], ['28,166.48', 'INVESTMENT_AMOUNT'], ['28,166.48', 'AMOUNT_DUE'], ['5,000,000.00', 'COMMITMENT_AMOUNT'], ['4,416,007.41', 'UNDRAWN_COMMITMENT_BEFORE'], ['4,416,007.41', 'UNFUNDED_COMMITMENT']]


Heading: 
MISC At Page 2




In [ ]:
!zip -r /content/model-best.zip /content/model-best


  adding: content/model-best/ (stored 0%)
  adding: content/model-best/config.cfg (deflated 61%)
  adding: content/model-best/tok2vec/ (stored 0%)
  adding: content/model-best/tok2vec/cfg (stored 0%)
  adding: content/model-best/tok2vec/model (deflated 8%)
  adding: content/model-best/vocab/ (stored 0%)
  adding: content/model-best/vocab/strings.json (deflated 72%)
  adding: content/model-best/vocab/lookups.bin (stored 0%)
  adding: content/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/model-best/vocab/key2row (stored 0%)
  adding: content/model-best/vocab/vectors (deflated 45%)
  adding: content/model-best/meta.json (deflated 77%)
  adding: content/model-best/tokenizer (deflated 81%)
  adding: content/model-best/ner/ (stored 0%)
  adding: content/model-best/ner/moves (deflated 82%)
  adding: content/model-best/ner/cfg (deflated 33%)
  adding: content/model-best/ner/model (deflated 13%)
